In [107]:
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)
pd.reset_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None) 

In [ ]:
ogair_df = pd.read_csv('C:/Users/rubyc/Documents/work/ebfd/requests_and_consultations_airtable_92724.csv')
ogair_df.info()

# 1. Data Cleaning

##1.1 Null Values
**Purpose:** Check for the proportion of null values in the data and
<br>**Code Breakdown by line:**
<br>1. Uses Pandas built-in isnull function to check for null values and calculate the number against the total number of values in the dataframe
<br> 2. Makes a dataframe of these proportions we just calculated
<br> 3. Sorts them by the highest to lowest proportion
<br> 4. Calls the dataframe we just made
<br> 5. Replaces all null values (which python reads as NaN) with the string "None"
<br>**Notes:** None this is very simple and obligatory. We chose to replace NaN with the string "None" bc most of what we're doing here is natural language processing, which utelizes functions that are designed to have a strings input and tend to output errors when they're input NaN instead.

In [ ]:
null_percentage = ogair_df.isnull().sum() / len(ogair_df)
missing_value_df = pd.DataFrame({'proportion_null': null_percentage})
missing_value_df = missing_value_df.sort_values('proportion_null', ascending = False)
missing_value_df.head(30)

In [ ]:
nullfree_df = ogair_df.dropna(subset='Date Called').fillna('None') #.dropna(subset=['Date Called','QLSP?'])
nullfree_df.info()

In [ ]:

test_df = nullfree_df.head(5)
test_df.head()

# 2. Parsing out the data

##2.1 Callback Dates/Date Processing
**Purpose:** Extract the date that the call was returned from the "Notes" section and convert the extracted callback date and native call recieved date
<br>**Input:** The name of a dataframe, the column that the noted dates are in, and the column that the full call cates are in
<br>**Output:** A list of the callback dates in datetime format
<br>**Code Breakdown:**
1.  Creates empty list to later fill
2. Two regular expressions are defined as "yearpattern" and "datepattern" - yearpattern identifies four numerical digits in a row and datepattern identifies numbers 1-12 immediately followed by a slash immediately followed by numbers 1-31. I have no idea how regular expression syntax works those were generated w Chatgpt
3. Opens a for loop that'll go through every row in the column that you've defined as containing the callback dates, defining the contents of each row as "note"
  1. Within the for loop, we search the contents of the
5. Replaces all null values (which python reads as NaN) with the string "None"
**Notes:** None this is very simple and obligatory. We chose to replace NaN with the string "None" bc most of what we're doing here is natural language processing, which utelizes functions that are designed to have a strings input and tend to output errors when they're input NaN instead.

In [102]:
def pull_callback_date(df, datecol, yearcol):
  #designed for input: nullfree_df, Notes, Date Called
  callback_date_l = []
  yearpattern = r'\d{4}'
  datepattern = r'\b(0?[1-9]|1[0-2])/(0?[1-9]|[12][0-9]|3[01])\b'
  for note in df[datecol]: #can i do for note, date in df[datecol], df[yearcol]??? 
    #  try:
    datefind = re.search(datepattern, note)
    if datefind:
      callbackdate = datefind.group()
      date = df[df[datecol]==note][yearcol].iloc[0] ##identifies the whole row of where note is a given note, so won't work if it's not unique 
      yearfind = re.search(yearpattern, date)
      if yearfind:
      #for date in df[yearcol]: #this for loop should not be nested here like this 
        wholedate = (callbackdate + "/" + yearfind.group())
        final_datetime = pd.to_datetime(wholedate)
      #print(yearfind)
        callback_date_l.append(final_datetime)
      else: callback_date_l.append(None)
    else: callback_date_l.append(None)
      #except Exception as e:
      #    print(f"I'm a dumb robot and couldn't FUCKING process {note}, because {e}")
  return callback_date_l

In [ ]:
for i in nullfree_df: 
    print(i)
    

In [ ]:
yearpattern = r'\d{4}'
yearfind = re.search(yearpattern, nullfree_df['Date Called'][1])
yearfind.group()

In [ ]:
nullfree_df['callback date datetime'] = pull_callback_date(nullfree_df, 'Notes', 'Date Called')
nullfree_df['call date datetime'] = pd.to_datetime(nullfree_df['Date Called'])
nullfree_df.info()
#nullfree_df['callback date datetime'] = pd.to_datetime(nullfree_df['callback date datetime'], errors='coerce')

In [67]:
#referral_org_s = nullfree_df.groupby('Referral From (Agency/Individual)').size()
#referral_org_s.to_frame().rename({0: "Count"}, axis = 1).reset_index().sort_values("Count", ascending = False)
#len(nullfree_df['Referral From (Agency/Individual)'].unique())

In [ ]:
nullfree_df['call date datetime'][34]

In [105]:
def calculate_date_differences(df):
    # Create a new column to store the date differences
    df['date_difference'] = None
    
    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        call_date = row['call date datetime']
        callback_date = row['callback date datetime']
        #try:
        if callback_date is None:
            continue
        #date_difference = (callback_date - call_date).days  # or use .total_seconds() for a more precise measure
        #df.at[index, 'date_difference'] = date_difference #stores it in a new column idk how this line works 
        print(call_date, callback_date)
        #except Exception as e:
            #print(f"I'm a dumb robot and couldn't FUCKING process {row}, because {e}")    

    return df

In [ ]:
calculate_date_differences(nullfree_df)


░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░█████████████░░░░░░░░░░░
░░░░░░░░░████████░░░░░░░░░░░█████░░░░░░░
░░░░░░███░░░░░░░░██████░░░░░░░░░██░░░░░░
░░░░░░█░░░░░░░░░░█░░░░█░░░░░░░░░░█░░░░░░
░░░░░██░░░░░░░░░░░░░░░░░░░░░░░░███░░░░░░
░░░░░░█░░░░░░░░░░░░░░░░░░░░░█████░░░░░░░
░░░░░█████░░░░░░░░░░░░░░██████░░█░░░░░░░
░░░░░░█░░███████████████░█░░░█░░█░░░░░░░
░░░░░░░█░░░░░█░░░░░█░░░░░█░░░█░░█░░░░░░░
░░░░░░░██░░░░█░░░░░█░░░░░█░░░█░█░░░░░░░░
░░░░░░░░█░░░░██░░░░█░░░░█░░░░█░█░░░░░░░░
░░░░░░░░░█░░░░█░░░░█░░░░█░░░█░██░░░░░░░░
░░░░░░░░░█░░░░░█░░░█░░░░█░░░█░█░░░░░░░░░
░░░░░░░░░██░░░░██░░█░░░██░░░█░█░░░░░░░░░
░░░░░░░░░░█░░░░░█░░██░░█░░░░█░█░░░░░░░░░
░░░░░░░░░░████░░█░░░█░░█░░░█░░█░░░░░░░░░
░░░░░░░░░░░░░██████████████████░░░░░░░░░
░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░


In [ ]:
pd.to_datetime(nullfree_df['Date Called'][2])

In [ ]:
def calldate_datetime(df, col):
  calldate_l = []
  for date in df[col]:
    calldatetime = pd.to_datetime(date)
    calldate_l.append(calldatetime)
  return calldate_l
## I don't think calldate_l ever gets put into the df

nullfree_df['callback date datetime'] = pull_callback_date(nullfree_df, 'Notes', 'Date Called')
nullfree_df['call date datetime'] = calldate_datetime(nullfree_df, 'Date Called')
#nullfree_df['callback time'] = nullfree_df['call date datetime'] - nullfree_df['callback date datetime']

In [ ]:
referrals_s = nullfree_df.groupby('Referral From (Agency/Individual)').size()
referrals_df = referrals_s.to_frame().rename({0: "Count"}, axis = 1).reset_index()
referrals_df.sort_values("Count", ascending = False)

In [ ]:
if date != 'None':
    calldate_l.append(pd.to_datetime(date))
else:
    calldate_l.append('None')

In [ ]:
nullfree_df['callback_date'] = pull_callback_date(nullfree_df, 'Notes', 'Date Called')
nullfree_df['Date Called Formatted'] = calldate_datetime(nullfree_df, 'Date Called')

In [ ]:
nullfree_df.to_csv('airtablewdates2.csv', index=True)  

In [ ]:
callback_date_l = []
yearpattern = r'\d{4}'
datepattern_old = r'\d{1,2}/\d{1,2}'
datepattern = r'\b(0?[1-9]|1[0-2])/(0?[1-9]|[12][0-9]|3[01])\b'
for note in nullfree_df['Notes']:
  #  try:
    #need to if/then for if it's empty empty
  datefind = re.search(datepattern, note)
  if datefind:
    callbackdate = datefind.group()
    for date in nullfree_df['Date Called']:
      yearfind = re.search(yearpattern, date)
    wholedate = (callbackdate + "/" + yearfind.group())
    final_datetime = pd.to_datetime(wholedate)
    callback_date_l.append(final_datetime)
    #except Exception as e:
    #    print(f"I'm a dumb robot and couldn't FUCKING process {note}, because {e}")
callback_date_l
#print(datefind)